# Hugging Face Local Pipelines | Hugging Face ローカルパイプライン

Hugging Faceのモデルは、`HuggingFacePipeline` クラスを通じてローカルで実行することができます。

> Hugging Face models can be run locally through the `HuggingFacePipeline` class.

[Hugging Face Model Hub](https://huggingface.co/models)は、12万以上のモデル、2万以上のデータセット、5万以上のデモアプリ（Spaces）をホストしており、すべてオープンソースで公開されているオンラインプラットフォームです。ここでは、人々が簡単に協力し、共に機械学習を構築できます。

> The [Hugging Face Model Hub](https://huggingface.co/models) hosts over 120k models, 20k datasets, and 50k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

これらは、このローカルパイプラインラッパーを通じて、またはHuggingFaceHubクラスを通じてホストされた推論エンドポイントを呼び出すことによってLangChainから呼び出すことができます。ホストされたパイプラインの詳細については、[HuggingFaceHub](huggingface_hub.html)ノートブックをご覧ください。

> These can be called from LangChain either through this local pipeline wrapper or by calling their hosted inference endpoints through the HuggingFaceHub class. For more information on the hosted pipelines, see the [HuggingFaceHub](huggingface_hub.html) notebook.



使用するには、`transformers` Pythonパッケージを[インストール](https://pypi.org/project/transformers/)しておく必要があり、また[pytorch](https://pytorch.org/get-started/locally/)もインストールしてください。さらにメモリ効率の良いアテンション実装のために、`xformer`をインストールすることもできます。

> To use, you should have the `transformers` python [package installed](https://pypi.org/project/transformers/), as well as [pytorch](https://pytorch.org/get-started/locally/). You can also install `xformer` for a more memory-efficient attention implementation.



In [ ]:
%pip install transformers --quiet

### Model Loading | モデルの読み込み

`from_model_id` メソッドを使用してモデルパラメータを指定することで、モデルをロードすることができます。

> Models can be loaded by specifying the model parameters using the `from_model_id` method.



In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
)

また、既存の `transformers` パイプラインを直接渡すことで読み込むこともできます

> They can also be loaded by passing in an existing `transformers` pipeline directly



In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

### Create Chain | チェーンを作成する

モデルをメモリにロードしたら、プロンプトと組み合わせてチェーンを形成することができます。

> With the model loaded into memory, you can compose it with a prompt to
> form a chain.



In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

### GPU Inference | GPUによる推論

GPUを搭載したマシンで実行する際には、`device=n` パラメータを指定して、モデルを特定のデバイスに割り当てることができます。デフォルトはCPUでの推論のため `-1` に設定されています。

> When running on a machine with GPU, you can specify the `device=n` parameter to put the model on the specified device.
> Defaults to `-1` for CPU inference.

複数のGPUを使用している場合や、モデルが単一のGPUに収まりきらないほど大きい場合、`device_map="auto"`を指定することができます。これを使用するには[Accelerate](https://huggingface.co/docs/accelerate/index)ライブラリが必要であり、モデルの重みをどのようにロードするかを自動的に判断します。

> If you have multiple-GPUs and/or the model is too large for a single GPU, you can specify `device_map="auto"`, which requires and uses the [Accelerate](https://huggingface.co/docs/accelerate/index) library to automatically determine how to load the model weights.

注意：`device`と`device_map`は同時に指定してはいけません。同時に指定すると予期しない動作が発生する可能性があります。

> *Note*: both `device` and `device_map` should not be specified together and can lead to unexpected behavior.



In [ ]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 10},
)

gpu_chain = prompt | gpu_llm

question = "What is electroencephalography?"

print(gpu_chain.invoke({"question": question}))

### Batch GPU Inference | バッチ GPU 推論

GPUを搭載したデバイスで実行している場合、バッチモードでGPU上で推論を実行することもできます。

> If running on a device with GPU, you can also run inference on the GPU in batch mode.



In [ ]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",
    task="text-generation",
    device=0,  # -1 for CPU
    batch_size=2,  # adjust as needed based on GPU map and model size.
    model_kwargs={"temperature": 0, "max_length": 64},
)

gpu_chain = prompt | gpu_llm.bind(stop=["\n\n"])

questions = []
for i in range(4):
    questions.append({"question": f"What is the number {i} in french?"})

answers = gpu_chain.batch(questions)
for answer in answers:
    print(answer)